In [ ]:
import os, itertools
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import gensim
import gensim.downloader as api
from sklearn.decomposition import PCA
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
from matplotlib.pyplot import text
from sklearn.metrics import silhouette_score, silhouette_samples
from numpy import linalg
import statistics


In [ ]:
def drop_duplicates(labeled_df):
    data_copy_df = labeled_df.copy()
    data_no_duplicates = data_copy_df.drop_duplicates(subset = ['text'])
    return data_no_duplicates

file_path_l = []

for root, dirs, files in os.walk('InternClustersTextAlexisNoSub'):
    for filename in files:
        file_path_l.append(os.path.join(root, filename))

text_l = []
for file_path in file_path_l:
    filey = open(file_path,encoding="mbcs")
    text_l.append(filey.read())
    filey.close()
types_l = []
for namey in file_path_l:
    splitted = namey.split('\\')
    types_l.append(splitted[1])
#print(types_l)

In [ ]:
dataset = api.load('text8')
#dataset = api.load("wiki-english-20171001")
data = [d for d in dataset]

def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
data_for_training = list(tagged_document(data))


In [ ]:
model = gensim.models.doc2vec.Doc2Vec(epochs=30)
model.build_vocab(data_for_training)
model.train(data_for_training, total_examples=model.corpus_count, epochs=model.epochs)


In [ ]:
def cosine(vector1,vector2):
    cosV12 = np.dot(vector1, vector2) / (linalg.norm(vector1) * linalg.norm(vector2))
    return cosV12

dataset = text_l
List_of_List_of_words = []
for x in range(len(dataset)):
   List_of_List_of_words.append(dataset[x].split())
#print(model.infer_vector(List_of_List_of_words[0]))
vector1 = model.infer_vector(List_of_List_of_words[0])
vector2 = model.infer_vector(List_of_List_of_words[9])
vector3 = model.infer_vector(List_of_List_of_words[-1])
print("Vectors in the same category:", cosine(vector1,vector2))
print("Vectors in different categories:", cosine(vector1,vector3))

In [ ]:
vectors = []
for i in range(len(List_of_List_of_words)):
    vectors.append(model.infer_vector(List_of_List_of_words[i]))
print(len(vectors))
score = silhouette_score(vectors, types_l)
scoreC = silhouette_score(vectors, types_l, metric = "cosine")
scoreM = silhouette_score(vectors, types_l, metric = "manhattan")
print("Full dimensions:", score)
print("Full dimensions:", scoreC)
print("Full dimensinos:", scoreM)

samples = silhouette_samples(vectors, types_l, metric = "cosine")

types = list(set(types_l))

dict = {}

for label in types:
    dict[label] = []

for i in range(len(types_l)):
    dict[types_l[i]].append(samples[i])

#print(dict)

for key, value in dict.items():
    print(key, ": ", statistics.mean(value))

'''
pca2 = PCA(2)
vectors2 = pca2.fit_transform(vectors)
score2 = silhouette_score(vectors2, types_l)
score2C = silhouette_score(vectors2, types_l, metric = "cosine")
score2M = silhouette_score(vectors2, types_l, metric = "manhattan")
print("2 dimensions:", score2)
print("2 dimensions:", score2C)
print("2 dimensinos:", score2M)

pca10 = PCA(10)
vectors10 = pca10.fit_transform(vectors)
score10 = silhouette_score(vectors10, types_l)
score10C = silhouette_score(vectors10, types_l, metric = "cosine")
score10M = silhouette_score(vectors10, types_l, metric = "manhattan")
print("10 dimensions:", score10)
print("10 dimensions:", score10C)
print("10 dimensinos:", score10M)
'''


In [ ]:
docGroups = []
for x in range(len(List_of_List_of_words)):
    vector1 = model.infer_vector(List_of_List_of_words[x])
    tempList = []
    for y in range(len(List_of_List_of_words)):
        vector2 = model.infer_vector(List_of_List_of_words[y])
        if(cosine(vector1, vector2) >.75):
            tempList.append(y)
    docGroups.append(tempList)

for i in docGroups:
    print(i)
